Single Race Exploratory Data Analysis -

The purpose of this notebook is to explore and wrangle data from the Fast-F1 API for a single race. This serves as a starting point to establish an organized workflow for data acquisition, preparation, and early exploration before scaling to a multi-race analysis. Descriptive statistics and preliminary visualizations will be generated to provide an initial understanding of the data, identify patterns or anomalies, and inform subsequent feature engineering and modeling decisions.

The code below adds the parent directory to Python’s module search path and configures logging to suppress all FastF1 logs below the warning level. This will enable subsequent code blocks that use imports to work seamlessly and keep my resulting code compilations clean and easy to read.

In [10]:
import sys
import os
import logging

root = os.path.abspath("..")
sys.path.append(root)

logging.getLogger('fastf1').setLevel(logging.WARNING)

In this section, I import Python libraries for data visualization, numerical analysis, and working with Pandas dataframes that the FastF1 API is primarily structured with. I also import custom functions and modules for preprocessing F1 data and constants. To support full visibility into the datasets without truncation, I configure Pandas display options to show all rows and columns.

In [11]:
from src.data import f1_data
from src.utils import f1_constants, f1_pandas_helpers
from src.preprocessing import telemetry_cleaning

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)  # reset_option to compact view
pd.set_option('display.max_columns', None)


The following code initializes a single F1 race session by defining parameters such as year, location, and session type. These values are passed into the custom F1Session class (from f1_data.py), which creates a session object built on top of Fast-F1. This object provides access to race data as well as custom functions I’ve implemented.

The session parameters were chosen to best match Tier 1 control qualities:

- Weather: Abu Dhabi (dry conditions)
- Max Speed: C5 Ultra Soft Tires
- Minimize Outliers: Qualifying Sessions (Q3 & Q2)
- Traffic: Avoid tow laps to avoid slipstream bias

In [16]:
year = 2024
grand_prix = f1_constants.F1Constants.LOCATIONS["Abu Dhabi"]
session_type = f1_constants.F1Constants.SESSIONS["Q"]

session = f1_data.F1Session(year, grand_prix, session_type)

All drivers who participated in the specific location's Q2 and Q3 sessions will be analyzed and assigned variables to be identified by their three-letter name code. Q3 data will be used for each driver whenever available; otherwise, Q2 data will be selected.

In [17]:
norris = f1_constants.F1Constants.DRIVERS["Lando Norris"]
piastri = f1_constants.F1Constants.DRIVERS["Oscar Piastri"]
verstappen = f1_constants.F1Constants.DRIVERS["Max Verstappen"]
perez = f1_constants.F1Constants.DRIVERS["Sergio Perez"]
sainz = f1_constants.F1Constants.DRIVERS["Carlos Sainz"]
leclerc = f1_constants.F1Constants.DRIVERS["Charles Leclerc"]
bottas = f1_constants.F1Constants.DRIVERS["Valtteri Bottas"]
alonso = f1_constants.F1Constants.DRIVERS["Fernando Alonso"]
stroll = f1_constants.F1Constants.DRIVERS["Lance Stroll"]
gasly = f1_constants.F1Constants.DRIVERS["Pierre Gasly"]
hulkenberg = f1_constants.F1Constants.DRIVERS["Nico Hulkenberg"]
magnussen = f1_constants.F1Constants.DRIVERS["Kevin Magnussen"]
lawson = f1_constants.F1Constants.DRIVERS["Liam Lawson"]
tsunoda = f1_constants.F1Constants.DRIVERS["Yuki Tsunoda"]
russell = f1_constants.F1Constants.DRIVERS["George Russell"]

For each driver, their fastest qualifying lap is identified by filtering timestamps for each sector. The resulting sector datasets are cleaned and then merged with other drivers’ telemetry data for that sector.

In [18]:
# Q3 data for Lando Norris, McLaren
q1, q2, q3 = session.get_laps(norris).split_qualifying_sessions()
norris_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:59.496000"
sector_1_end = "0 days 01:17:16.260000"
sector_2_end = "0 days 01:17:52.036000"
sector_3_end = "0 days 01:18:21.897000"

norris_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_1_start, end=sector_1_end)
norris_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(norris_sector1_telemetry, norris)

# find_steering_wheel_angle()

norris_sector1_telemetry_cleaned

# norris_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_1_end, end=sector_2_end)
# norris_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(norris_sector2_telemetry, norris)
# norris_sector2_telemetry_cleaned

# norris_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_2_end, end=sector_3_end)
# norris_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(norris_sector3_telemetry, norris)
# norris_sector3_telemetry_cleaned

,DriverCode,SessionTime (s),SectorTime (s),RPM,Speed (m/s),nGear,Throttle (%),BrakesApplied,Distance (m),X (1/10 m),Y (1/10 m),Z (1/10 m)
7979,NOR,01:16:59.557,00:17:19.828,11229.000000,68.333333,6,100.000000,False,20968.508889,778.569904,2105.751713,-239.001361
7980,NOR,01:16:59.654,00:17:19.925,11318.724941,68.932098,6,100.000000,False,20975.275273,844.000000,2113.000000,-239.000000
7981,NOR,01:16:59.874,00:17:20.145,11522.224941,70.290123,6,100.000000,False,20990.849977,997.000000,2130.000000,-239.000000
7982,NOR,01:16:59.917,00:17:20.188,11562.000000,70.555556,6,100.000000,False,20993.908889,1026.891384,2133.312099,-238.974188
7983,NOR,01:17:00.055,00:17:20.326,11665.883237,70.981481,6,100.000000,False,21003.748877,1123.000000,2144.000000,-239.000000
7984,NOR,01:17:00.274,00:17:20.545,11830.741618,71.657407,6,100.000000,False,21019.491494,1280.000000,2162.000000,-240.000000
7985,NOR,01:17:00.277,00:17:20.548,11833.000000,71.666667,6,100.000000,False,21019.708889,1282.167908,2162.253093,-240.006256
7986,NOR,01:17:00.474,00:17:20.745,11091.172116,72.692708,6,100.000000,False,21034.089617,1425.000000,2179.000000,-240.000000
7987,NOR,01:17:00.597,00:17:20.868,10628.000000,73.333333,7,100.000000,False,21043.175556,1514.747481,2189.147597,-240.558748
7988,NOR,01:17:00.693,00:17:20.964,10657.866667,73.777778,7,100.000000,False,21050.324912,1585.000000,2197.000000,-241.000000


In [ ]:
# Q3 data for Oscar Piastri, McLaren
q1, q2, q3 = session.get_laps(piastri).split_qualifying_sessions()
piastri_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:27.506000"
sector_1_end = "0 days 01:16:44.333000"
sector_2_end = "0 days 01:17:20.186000"
sector_3_end = "0 days 01:17:50.110000"

piastri_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_1_start, end=sector_1_end)
piastri_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(piastri_sector1_telemetry, piastri)
piastri_sector1_telemetry_cleaned

# piastri_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_1_end, end=sector_2_end)
# piastri_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(piastri_sector2_telemetry, piastri)
# piastri_sector2_telemetry_cleaned

# piastri_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_2_end, end=sector_3_end)
# piastri_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(piastri_sector3_telemetry, piastri)
# piastri_sector3_telemetry_cleaned

In [ ]:
# Q3 data for Max Verstappen, Red Bull
q1, q2, q3 = session.get_laps(verstappen).split_qualifying_sessions()
verstappen_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:08:27.031000"
sector_1_end = "0 days 01:08:44.036000"
sector_2_end = "0 days 01:09:19.766000"
sector_3_end = "0 days 01:09:49.959000"

verstappen_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_1_start, end=sector_1_end)
verstappen_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(verstappen_sector1_telemetry, verstappen)
verstappen_sector1_telemetry_cleaned

# verstappen_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_1_end, end=sector_2_end)
# verstappen_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(verstappen_sector2_telemetry, verstappen)
# verstappen_sector2_telemetry_cleaned

# verstappen_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_2_end, end=sector_3_end)
# verstappen_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(verstappen_sector3_telemetry, verstappen)
# verstappen_sector3_telemetry_cleaned


In [ ]:
# Q3 data for Sergio Perez, Red Bull
q1, q2, q3 = session.get_laps(perez).split_qualifying_sessions()
perez_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:14:50.040000"
sector_1_end = "0 days 01:15:06.945000"
sector_2_end = "0 days 01:15:42.890000"
sector_3_end = "0 days 01:16:13.127000"

perez_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_1_start, end=sector_1_end)
perez_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(perez_sector1_telemetry, perez)
perez_sector1_telemetry_cleaned

# perez_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_1_end, end=sector_2_end)
# perez_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(perez_sector2_telemetry, perez)
# perez_sector2_telemetry_cleaned

# perez_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_2_end, end=sector_3_end)
# perez_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(perez_sector3_telemetry, perez)
# perez_sector3_telemetry_cleaned

In [ ]:
# Q3 data for Carlos Sainz, Ferrari
q1, q2, q3 = session.get_laps(sainz).split_qualifying_sessions()
sainz_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:53.219000"
sector_1_end = "0 days 01:17:10.014000"
sector_2_end = "0 days 01:17:45.801000"
sector_3_end = "0 days 01:18:15.858000"

sainz_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_1_start, end=sector_1_end)
sainz_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(sainz_sector1_telemetry, sainz)
sainz_sector1_telemetry_cleaned

# sainz_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_1_end, end=sector_2_end)
# sainz_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(sainz_sector2_telemetry, sainz)
# sainz_sector2_telemetry_cleaned

# sainz_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_2_end, end=sector_3_end)
# sainz_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(sainz_sector3_telemetry, sainz)
# sainz_sector3_telemetry_cleaned

In [ ]:
# Q2 data for Charles Leclerc, Ferrari
q1, q2, q3 = session.get_laps(leclerc).split_qualifying_sessions()
leclerc_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:03.473000"
sector_1_end = "0 days 00:57:20.332000"
sector_2_end = "0 days 00:57:56.296000"
sector_3_end = "0 days 00:58:26.338000"

leclerc_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_1_start, end=sector_1_end)
leclerc_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(leclerc_sector1_telemetry, leclerc)
leclerc_sector1_telemetry_cleaned

# leclerc_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_1_end, end=sector_2_end)
# leclerc_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(leclerc_sector2_telemetry, leclerc)
# leclerc_sector2_telemetry_cleaned

# leclerc_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_2_end, end=sector_3_end)
# leclerc_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(leclerc_sector3_telemetry, leclerc)
# leclerc_sector3_telemetry_cleaned

In [ ]:
# Q3 data for Valtteri Bottas, Alfa Romeo
q1, q2, q3 = session.get_laps(bottas).split_qualifying_sessions()
bottas_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:15:46.639000"
sector_1_end = "0 days 01:16:03.650000"
sector_2_end = "0 days 01:16:39.552000"
sector_3_end = "0 days 01:17:09.702000"

bottas_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_1_start, end=sector_1_end)
bottas_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(bottas_sector1_telemetry, bottas)
bottas_sector1_telemetry_cleaned

# bottas_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_1_end, end=sector_2_end)
# bottas_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(bottas_sector2_telemetry, bottas)
# bottas_sector2_telemetry_cleaned

# bottas_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_2_end, end=sector_3_end)
# bottas_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(bottas_sector3_telemetry, bottas)
# bottas_sector3_telemetry_cleaned

In [ ]:
# Q3 data for Fernando Alonso, Aston Martin
q1, q2, q3 = session.get_laps(alonso).split_qualifying_sessions()
alonso_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:45.371000"
sector_1_end = "0 days 01:17:02.191000"
sector_2_end = "0 days 01:17:38.197000"
sector_3_end = "0 days 01:18:08.377000"

alonso_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_1_start, end=sector_1_end)
alonso_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(alonso_sector1_telemetry, alonso)
alonso_sector1_telemetry_cleaned

# alonso_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_1_end, end=sector_2_end)
# alonso_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(alonso_sector2_telemetry, alonso)
# alonso_sector2_telemetry_cleaned

# alonso_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_2_end, end=sector_3_end)
# alonso_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(alonso_sector3_telemetry, alonso)
# alonso_sector3_telemetry_cleaned

In [ ]:
# Q2 data for Lance Stroll, Aston Martin
q1, q2, q3 = session.get_laps(stroll).split_qualifying_sessions()
stroll_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:56:38.573000"
sector_1_end = "0 days 00:56:55.483000"
sector_2_end = "0 days 00:57:31.468000"
sector_3_end = "0 days 00:58:02.147000"

stroll_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_1_start, end=sector_1_end)
stroll_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(stroll_sector1_telemetry, stroll)
stroll_sector1_telemetry_cleaned

# stroll_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_1_end, end=sector_2_end)
# stroll_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(stroll_sector2_telemetry, stroll)
# stroll_sector2_telemetry_cleaned

# stroll_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_2_end, end=sector_3_end)
# stroll_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(stroll_sector3_telemetry, stroll)
# stroll_sector3_telemetry_cleaned

In [ ]:
# Q3 data for Pierre Gasly, Alpine
q1, q2, q3 = session.get_laps(gasly).split_qualifying_sessions()
gasly_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:18.040000"
sector_1_end = "0 days 01:16:34.989000"
sector_2_end = "0 days 01:17:10.860000"
sector_3_end = "0 days 01:17:40.892000"

gasly_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_1_start, end=sector_1_end)
gasly_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(gasly_sector1_telemetry, gasly)
gasly_sector1_telemetry_cleaned

# gasly_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_1_end, end=sector_2_end)
# gasly_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(gasly_sector2_telemetry, gasly)
# gasly_sector2_telemetry_cleaned

# gasly_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_2_end, end=sector_3_end)
# gasly_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(gasly_sector3_telemetry, gasly)
# gasly_sector3_telemetry_cleaned

In [ ]:
# Q3 data for Nico Hulkenberg, Haas
q1, q2, q3 = session.get_laps(hulkenberg).split_qualifying_sessions()
hulkenberg_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:03.696000"
sector_1_end = "0 days 01:16:20.651000"
sector_2_end = "0 days 01:16:56.369000"
sector_3_end = "0 days 01:17:26.431000"

hulkenberg_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_1_start, end=sector_1_end)
hulkenberg_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(hulkenberg_sector1_telemetry, hulkenberg)
hulkenberg_sector1_telemetry_cleaned

# hulkenberg_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_1_end, end=sector_2_end)
# hulkenberg_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(hulkenberg_sector2_telemetry, hulkenberg)
# hulkenberg_sector2_telemetry_cleaned

# hulkenberg_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_2_end, end=sector_3_end)
# hulkenberg_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(hulkenberg_sector3_telemetry, hulkenberg)
# hulkenberg_sector3_telemetry_cleaned

In [ ]:
# Q2 data for Kevin Magnussen, Haas
q1, q2, q3 = session.get_laps(magnussen).split_qualifying_sessions()
magnussen_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:55:34.175000"
sector_1_end = "0 days 00:55:51.220000"
sector_2_end = "0 days 00:56:27.265000"
sector_3_end = "0 days 00:56:57.907000"

magnussen_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_1_start, end=sector_1_end)
magnussen_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(magnussen_sector1_telemetry, magnussen)
magnussen_sector1_telemetry_cleaned

# magnussen_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_1_end, end=sector_2_end)
# magnussen_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(magnussen_sector2_telemetry, magnussen)
# magnussen_sector2_telemetry_cleaned

# magnussen_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_2_end, end=sector_3_end)
# magnussen_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(magnussen_sector3_telemetry, magnussen)
# magnussen_sector3_telemetry_cleaned

In [ ]:
# Q2 data for Liam Lawson, Visa CashApp
q1, q2, q3 = session.get_laps(lawson).split_qualifying_sessions()
lawson_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:38.695000"
sector_1_end = "0 days 00:57:55.568000"
sector_2_end = "0 days 00:58:31.626000"
sector_3_end = "0 days 00:59:01.907000"

lawson_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_1_start, end=sector_1_end)
lawson_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(lawson_sector1_telemetry, lawson)
lawson_sector1_telemetry_cleaned

# lawson_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_1_end, end=sector_2_end)
# lawson_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(lawson_sector2_telemetry, lawson)
# lawson_sector2_telemetry_cleaned

# lawson_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_2_end, end=sector_3_end)
# lawson_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(lawson_sector3_telemetry, lawson)
# lawson_sector3_telemetry_cleaned

In [ ]:
# Q2 data for Yuki Tsunoda, Visa CashApp
q1, q2, q3 = session.get_laps(tsunoda).split_qualifying_sessions()
tsunoda_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:26.587000"
sector_1_end = "0 days 00:57:43.456000"
sector_2_end = "0 days 00:58:19.489000"
sector_3_end = "0 days 00:58:49.753000"

tsunoda_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_1_start, end=sector_1_end)
tsunoda_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(tsunoda_sector1_telemetry, tsunoda)
tsunoda_sector1_telemetry_cleaned

# tsunoda_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_1_end, end=sector_2_end)
# tsunoda_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(tsunoda_sector2_telemetry, tsunoda)
# tsunoda_sector2_telemetry_cleaned

# tsunoda_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_2_end, end=sector_3_end)
# tsunoda_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(tsunoda_sector3_telemetry, tsunoda)
# tsunoda_sector3_telemetry_cleaned

In [ ]:
# Q3 data for George Russell, Mercedes
q1, q2, q3 = session.get_laps(russell).split_qualifying_sessions()
russell_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:17:14.677000"
sector_1_end = "0 days 01:17:31.522000"
sector_2_end = "0 days 01:18:07.527000"
sector_3_end = "0 days 01:18:37.566000"

russell_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_1_start, end=sector_1_end)
russell_sector1_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(russell_sector1_telemetry, russell)
russell_sector1_telemetry_cleaned

# russell_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_1_end, end=sector_2_end)
# russell_sector2_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(russell_sector2_telemetry, russell)
# russell_sector2_telemetry_cleaned

# russell_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_2_end, end=sector_3_end)
# russell_sector3_telemetry_cleaned = telemetry_cleaning.clean_driver_sector_telemetry(russell_sector3_telemetry, russell)
# russell_sector3_telemetry_cleaned

Sector 1 telemetry data for all drivers is concatanated into a single dataframe below.

In [ ]:
total_sector1_telemetry = pd.concat([
    norris_sector1_telemetry_cleaned, 
    piastri_sector1_telemetry_cleaned, 
    verstappen_sector1_telemetry_cleaned, 
    perez_sector1_telemetry_cleaned,
    sainz_sector1_telemetry_cleaned,
    leclerc_sector1_telemetry_cleaned,
    bottas_sector1_telemetry_cleaned,
    alonso_sector1_telemetry_cleaned,
    stroll_sector1_telemetry_cleaned,
    gasly_sector1_telemetry_cleaned,
    hulkenberg_sector1_telemetry_cleaned,
    magnussen_sector1_telemetry_cleaned,
    lawson_sector1_telemetry_cleaned,
    tsunoda_sector1_telemetry_cleaned,
    russell_sector1_telemetry_cleaned
], axis=0)

total_sector1_telemetry

Sector 2 telemetry data for all drivers is concatenated into a single dataframe below.

In [ ]:
total_sector2_telemetry = pd.concat([
    norris_sector2_telemetry_cleaned, 
    piastri_sector2_telemetry_cleaned, 
    verstappen_sector2_telemetry_cleaned, 
    perez_sector2_telemetry_cleaned,
    sainz_sector2_telemetry_cleaned,
    leclerc_sector2_telemetry_cleaned,
    bottas_sector2_telemetry_cleaned,
    alonso_sector2_telemetry_cleaned,
    stroll_sector2_telemetry_cleaned,
    gasly_sector2_telemetry_cleaned,
    hulkenberg_sector2_telemetry_cleaned,
    magnussen_sector2_telemetry_cleaned,
    lawson_sector2_telemetry_cleaned,
    tsunoda_sector2_telemetry_cleaned,
    russell_sector2_telemetry_cleaned
], axis=0)

total_sector2_telemetry

Sector 3 telemetry data for all drivers is concatanated into a single dataframe below.

In [ ]:
total_sector3_telemetry = pd.concat([
    norris_sector3_telemetry_cleaned, 
    piastri_sector3_telemetry_cleaned, 
    verstappen_sector3_telemetry_cleaned, 
    perez_sector3_telemetry_cleaned,
    sainz_sector3_telemetry_cleaned,
    leclerc_sector3_telemetry_cleaned,
    bottas_sector3_telemetry_cleaned,
    alonso_sector3_telemetry_cleaned,
    stroll_sector3_telemetry_cleaned,
    gasly_sector3_telemetry_cleaned,
    hulkenberg_sector3_telemetry_cleaned,
    magnussen_sector3_telemetry_cleaned,
    lawson_sector3_telemetry_cleaned,
    tsunoda_sector3_telemetry_cleaned,
    russell_sector3_telemetry_cleaned
], axis=0)

total_sector3_telemetry